# Дообучение ModernBERT на мультилейбл датасете 🚀

Этот ноутбук содержит код для дообучения модели **ModernBERT** на задаче многометочной классификации текстов. Модель обучается на датасете, содержащем тексты и соответствующие им бинарные метки, которые указывают на наличие или отсутствие определенных категорий.

## 📌 Основные особенности

- 🧠 **Модель**: ModernBERT (современная версия BERT с улучшенной архитектурой)
- 🏷️ **Задача**: Многометочная классификация текстов
- 🚀 **Оптимизация**: Использование современных техник обучения, таких как:
  - Гибридное внимание (локальное + глобальное)
  - Rotary Position Embedding (RoPE) для длинных контекстов
  - GeGLU активация для улучшения нелинейных возможностей модели
- 💻 **Ускорение**: Поддержка смешанной точности (FP16) для ускорения обучения

## 📊 Описание датасета

### Структура датасета
- **Количество записей**: 35,303
- **Количество столбцов**: 20

### Названия столбцов
1. `текст` - Текст для классификации
2. `офлайн_преступления` - Признак, указывающий на упоминание офлайн-преступлений
3. `онлайн_преступления` - Признак, указывающий на упоминание онлайн-преступлений
4. `наркотики` - Признак, указывающий на упоминание наркотиков
5. `азартные_игры` - Признак, указывающий на упоминание азартных игр
6. `порнография` - Признак, указывающий на упоминание порнографии
7. `проституция` - Признак, указывающий на упоминание проституции
8. `рабство` - Признак, указывающий на упоминание рабства
9. `самоубийство` - Признак, указывающий на упоминание самоубийства
10. `терроризм` - Признак, указывающий на упоминание терроризма
11. `оружие` - Признак, указывающий на упоминание оружия
12. `бодишейминг` - Признак, указывающий на упоминание бодишейминга
13. `хейтспич` - Признак, указывающий на упоминание хейтспича
14. `политика` - Признак, указывающий на упоминание политики
15. `расизм` - Признак, указывающий на упоминание расизма
16. `религия` - Признак, указывающий на упоминание религии
17. `сексуальные_меньшинства` - Признак, указывающий на упоминание сексуальных меньшинств
18. `сексизм` - Признак, указывающий на упоминание сексизма
19. `социальная_несправедливость` - Признак, указывающий на упоминание социальной несправедливости
20. `нейтральный` - Признак, указывающий на нейтральный текст

### Типы данных
- **Текст**: `object` (1 колонка)
- **Метки**: `float64` (19 колонок)

## 🛠️ Основные шаги

1. **Загрузка данных**: Загрузка и предобработка датасета.
2. **Токенизация**: Преобразование текстов в формат, подходящий для модели.
3. **Создание модели**: Инициализация модели ModernBERT с настройкой для многометочной классификации.
4. **Обучение**: Обучение модели с использованием современных техник оптимизации.
5. **Оценка**: Вычисление метрик качества на валидационной выборке.
6. **Сохранение модели**: Сохранение обученной модели для последующего использования.

## 📊 Метрики качества

В процессе обучения вычисляются следующие метрики:
- **Hamming Loss**: Средняя ошибка классификации по всем меткам.
- **Subset Accuracy**: Точность классификации всех меток одновременно.
- **Macro Precision/Recall/F1**: Средние значения precision, recall и F1 по всем меткам.
- **Micro F1**: F1-мера, вычисленная с учетом всех предсказаний и истинных меток.
- **Jaccard Score**: Коэффициент сходства между предсказанными и истинными метками.

## 🚀 Запуск

Для запуска обучения необходимо:
1. Убедиться, что все зависимости установлены.
2. Указать путь к датасету в конфигурации.
3. Запустить ноутбук и дождаться завершения обучения.

---

<p align="center">⚡ Преобразите ваш NLP-пайплайн с ModernBERT уже сегодня!</p>

In [ ]:
!pip install -q numpy==1.26
!pip install -q packaging
!pip install -q torch==2.1.0 torchvision==0.16.0
!pip install -q setuptools scikit-learn
!pip install --upgrade -q  datasets==3.1.0 accelerate==1.2.1
!pip install -q "git+https://github.com/huggingface/transformers.git@6e0515e99c39444caae39472ee1b2fd76ece32f1" --upgrade

In [2]:
!nvidia-smi

Fri Jan 31 10:27:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:81:00.0 Off |                    0 |
| N/A   49C    P0             408W / 700W |  76034MiB / 81559MiB |     92%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Стандартные библиотеки Python
from datetime import datetime
import json
import os
from pathlib import Path

# Библиотеки для работы с данными
import numpy as np
import pandas as pd
from sklearn.metrics import (
    classification_report,
    hamming_loss,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score
)
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# PyTorch и связанные библиотеки
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

# Typing
from typing import Dict, List, Optional, Tuple, Union, Any

# Настройка CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
torch._dynamo.config.disable = True


class MultiLabelTextDataset(Dataset):
    """
    Description:
        Пользовательский класс датасета для многометочной классификации текстов.
        
    Args:
        texts: Список текстов для классификации
        labels: Массив меток для каждого текста
        tokenizer: Токенизатор для предобработки текстов
        max_length: Максимальная длина токенизированного текста
        
    Returns:
        Объект Dataset для использования с DataLoader
        
    Examples:
        >>> dataset = MultiLabelTextDataset(
        ...     texts=['текст1', 'текст2'],
        ...     labels=np.array([[1, 0], [0, 1]]),
        ...     tokenizer=tokenizer,
        ...     max_length=512
        ... )
    """
    
    def __init__(
        self,
        texts: List[str],
        labels: np.ndarray,
        tokenizer: AutoTokenizer,
        max_length: int = 512
    ) -> None:
        # Сохраняем входные данные как атрибуты класса для использования в __getitem__
        self.texts      = texts
        self.labels     = labels
        self.tokenizer  = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        # Возвращает количество примеров в датасете (количество текстов)
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Получаем текст и метки по индексу
        text = str(self.texts[idx])
        # Преобразуем метки в float32 для обучения нейронной сети
        labels = self.labels[idx].astype(np.float32)

        # Токенизируем текст
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,     # Добавляем специальные токены ([CLS], [SEP] и т.д.)
            max_length=self.max_length,  # Ограничиваем длину последовательности
            padding='max_length',        # Дополняем последовательность до max_length
            truncation=True,             # Обрезаем тексты, превышающие max_length
            return_tensors='pt'          # Возвращаем тензоры PyTorch вместо списков
        )

        # Возвращаем словарь с входными данными и метками
        return {
            # Преобразуем тензор размерности (1, max_length) в (max_length,)
            'input_ids': encoding['input_ids'].flatten(),
            
            # attention_mask: 1 для настоящих токенов, 0 для padding токенов
            'attention_mask': encoding['attention_mask'].flatten(),
            
            # Преобразуем numpy массив меток в тензор PyTorch
            'labels': torch.FloatTensor(labels)
        }


def load_data(
    data_path: str,
    text_column: str = 'текст'
) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Description:
        Загрузка и предобработка датасета из CSV файла.
        
    Args:
        data_path: Путь к CSV файлу с данными
        text_column: Название колонки с текстами
        
    Returns:
        Кортеж (тексты, метки, названия_меток)
        
    Raises:
        FileNotFoundError: Если файл не найден
    """
    print(f"\nЗагрузка данных из {data_path}...")
    df = pd.read_csv(data_path)
    
    # Разделение признаков и меток
    texts = df[text_column].values
    label_columns = [col for col in df.columns if col != text_column]
    labels = df[label_columns].values
    
    print(f"Загружено {len(texts)} записей с {len(label_columns)} метками")
    return texts, labels, label_columns


def evaluate_multilabel(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    label_columns: Optional[List[str]] = None
) -> Dict[str, Union[float, Dict[str, Dict[str, float]]]]:
    """
    Description:
        Вычисляет метрики для мультилейбл классификации.
        
    Args:
        y_true: Истинные метки
        y_pred: Предсказанные метки
        label_columns: Названия меток для детальной оценки
        
    Returns:
        Словарь с общими метриками и метриками по каждой метке
    """
    # Общие метрики для мультилейбл классификации
    metrics = {
        'hamming_loss':    hamming_loss(y_true, y_pred),
        'subset_accuracy': accuracy_score(y_true, y_pred),
        'macro_precision': precision_score(y_true, y_pred, average='macro'),
        'macro_recall':    recall_score(y_true, y_pred, average='macro'),
        'macro_f1':        f1_score(y_true, y_pred, average='macro'),
        'micro_f1':        f1_score(y_true, y_pred, average='micro'),
        'samples_f1':      f1_score(y_true, y_pred, average='samples'),
        'jaccard_score':   jaccard_score(y_true, y_pred, average='samples')
    }
    
    # Метрики по отдельным меткам
    if label_columns is not None:
        per_label_metrics = {}
        for i, label in enumerate(label_columns):
            per_label_metrics[label] = {
                'precision': precision_score(y_true[:, i], y_pred[:, i]),
                'recall': recall_score(y_true[:, i], y_pred[:, i]),
                'f1': f1_score(y_true[:, i], y_pred[:, i])
            }
        metrics['per_label'] = per_label_metrics
    
    return metrics


class MultilabelClassifier(nn.Module):
    """
    Description:
        Модель многометочной классификации на основе ModernBERT.

    Args:
        model_name (str): Название предобученной модели
        num_labels (int): Количество меток для классификации

    Returns:
        None

    Raises:
        None

    Examples:
        >>> model = MultilabelClassifier("bert-base-uncased", 3)
        >>> input_ids = torch.tensor([[101, 2054, 2003, 102]])
        >>> attention_mask = torch.tensor([[1, 1, 1, 1]])
        >>> labels = torch.tensor([[1, 0, 1]])
        >>> outputs = model(input_ids, attention_mask, labels)
    """

    def __init__(self, model_name: str, num_labels: int) -> None:
        super().__init__()
        
        # Загружаем предобученную модель BERT с настройкой для многометочной классификации
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,                      # Устанавливаем количество выходных меток
            problem_type="multi_label_classification",  # Указываем тип задачи
            ignore_mismatched_sizes=True                # Игнорируем несовпадение размеров при загрузке
        )
        # Переопределяем классификационный слой
        # Это необходимо для корректной инициализации весов для нашей задачи
        self.bert.classifier = nn.Linear(
            self.bert.config.hidden_size,               # Размерность выхода BERT
            num_labels                                  # Количество классов для предсказания
        )

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        labels: Optional[torch.Tensor] = None
    ) -> Any:
        """
        Description:
            Выполняет прямой проход через модель.

        Args:
            input_ids (torch.Tensor): Тензор идентификаторов входных токенов.
            attention_mask (torch.Tensor): Тензор маски внимания.
            labels (Optional[torch.Tensor]): Тензор меток (опционально).

        Returns:
            Any: Выход модели.

        Raises:
            None

        Examples:
            >>> input_ids = torch.tensor([[101, 2054, 2003, 102]])
            >>> attention_mask = torch.tensor([[1, 1, 1, 1]])
            >>> labels = torch.tensor([[1, 0, 1]])
            >>> outputs = model.forward(input_ids, attention_mask, labels)
        """
        # Пропускаем данные через BERT и получаем выходы
        outputs = self.bert(
            input_ids=input_ids,            # Токенизированный текст
            attention_mask=attention_mask,  # Маска для различения реальных токенов от padding
            labels=labels                   # Если переданы метки, вычисляется BCE loss
        )
        return outputs


def train_model(
    model: MultilabelClassifier,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler._LRScheduler,
    device: torch.device,
    num_epochs: int,
    patience: int,
    output_dir: Path,
    label_columns: List[str]
) -> MultilabelClassifier:
    """
    Description:
        Функция обучения модели с логированием.
        
    Args:
        model: Модель для обучения
        train_loader: Загрузчик обучающих данных
        val_loader: Загрузчик валидационных данных
        optimizer: Оптимизатор
        scheduler: Планировщик скорости обучения
        device: Устройство для обучения (CPU/GPU)
        num_epochs: Количество эпох обучения
        patience: Количество эпох для ранней остановки
        output_dir: Директория для сохранения результатов
        label_columns: Названия меток
        
    Returns:
        Обученная модель
    """
    # Инициализация функции потерь для многометочной классификации
    criterion = nn.BCEWithLogitsLoss()
    
    # Инициализация переменных для отслеживания процесса обучения
    best_val_loss = float('inf')   # Лучшее значение ошибки на валидации
    early_stopping_counter = 0     # Счетчик эпох без улучшения
    
    # Словарь для хранения истории обучения
    history = {
        'train_loss': [],    # История ошибок на обучающей выборке
        'val_loss': [],      # История ошибок на валидационной выборке
        'metrics': []        # История метрик качества
    }
    
    print("\nНачало обучения:")
    print("-" * 50)
    
    # Основной цикл обучения по эпохам
    for epoch in range(num_epochs):
        print(f"\nЭпоха {epoch + 1}/{num_epochs}")
        print("=" * 50)
        
        # ===================== ФАЗА ОБУЧЕНИЯ =====================
        model.train()  # Переключение модели в режим обучения
        train_loss = 0
        
        print("\nОбучение:")
        for batch in tqdm(train_loader, desc='Прогресс'):
            # Обнуление градиентов перед каждым батчем
            optimizer.zero_grad()
            
            # Перемещение данных на целевое устройство
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Прямой проход
            outputs = model(input_ids, attention_mask)
            # Вычисление функции потерь
            loss = criterion(outputs.logits, labels)
            
            # Обратный проход и оптимизация
            loss.backward()
            # Ограничение градиентов для стабильности обучения
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        # Вычисление средней ошибки за эпоху
        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)
        
        # ===================== ФАЗА ВАЛИДАЦИИ =====================
        model.eval()     # Переключение модели в режим оценки
        val_loss = 0
        all_preds = []   # Список для сбора всех предсказаний
        all_labels = []  # Список для сбора всех истинных меток
        
        print("\nВалидация:")
        with torch.no_grad():  # Отключение вычисления градиентов
            for batch in tqdm(val_loader, desc='Прогресс'):
                # Перемещение данных на целевое устройство
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                # Получение предсказаний модели
                outputs = model(input_ids, attention_mask)
                # Вычисление функции потерь
                loss = criterion(outputs.logits, labels)
                
                val_loss += loss.item()
                
                # Преобразование логитов в вероятности и сбор предсказаний
                preds = torch.sigmoid(outputs.logits).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
        
        # Вычисление средней ошибки валидации
        avg_val_loss = val_loss / len(val_loader)
        history['val_loss'].append(avg_val_loss)
        
        # ===================== ВЫЧИСЛЕНИЕ МЕТРИК =====================
        # Преобразование предсказаний в бинарные метки
        all_preds = np.array(all_preds) > 0.5
        all_labels = np.array(all_labels)
        
        # Расчет метрик качества для всех меток
        metrics = evaluate_multilabel(all_labels, all_preds, label_columns)
        history['metrics'].append(metrics)
        
        # Вывод общих метрик
        print("\nОбщие метрики мультилейбл классификации:")
        print("-" * 50)
        for metric_name, value in metrics.items():
            if metric_name != 'per_label':
                print(f"{metric_name:20s}: {value:.4f}")
        
        # Вывод метрик по каждой метке отдельно
        print("\nМетрики по отдельным меткам:")
        print("-" * 50)
        for label, label_metrics in metrics['per_label'].items():
            print(f"{label:25s}:", end=" ")
            print(", ".join([
                f"{k}: {v:.4f}"
                for k, v in label_metrics.items()
            ]))
        
        # Вывод результатов эпохи
        print("\nРезультаты эпохи:")
        print(f"Средняя ошибка обучения:    {avg_train_loss:.4f}")
        print(f"Средняя ошибка валидации:   {avg_val_loss:.4f}")
        
        # ===================== РАННЯЯ ОСТАНОВКА =====================
        # Проверка на улучшение результатов
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
            
            # Сохранение лучшей версии модели
            print(f"\nСохранение лучшей модели в {output_dir}/best_model")
            model.bert.save_pretrained(Path(output_dir) / "best_model")
        else:
            # Увеличение счетчика эпох без улучшения
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("\nРанняя остановка!")
                break
        
        # Обновление learning rate по расписанию
        scheduler.step()
    
    # Сохранение истории обучения в JSON формате
    history_path = Path(output_dir) / "training_history.json"
    print(f"\nСохранение истории обучения в {history_path}")
    with open(history_path, "w") as f:
        json.dump(history, f)
    
    return model


def main() -> None:
    """
    Description:
        Основная функция для обучения модели многометочной классификации.
    """
    # Конфигурация
    CONFIG = {
        # Базовые параметры в первом приближении
        'model_id': "modernBERT-large-multilingual",          # Идентификатор модели
        'data_path': "data/dataset.csv",                      # Путь к данным
        'output_dir': "modernbert-russian-multilabel-final",  # Директория для вывода результатов
        'max_length': 1024,                                   # Максимальная длина последовательности
        'num_epochs': 5,                                      # Количество эпох для обучения
        'patience': 3,                                        # Параметр терпения для ранней остановки
        'learning_rate': 2e-5,                                # Скорость обучения
        'weight_decay': 0.01,                                 # Коэффициент регуляризации (L2)
        'train_batch_size': 16,                               # Размер батча для обучения
        'eval_batch_size': 32,                                # Размер батча для оценки
    
        # Новые гиперпараметры (эмпирическое приближение)
        'gradient_clip_max_norm': 1.0,                        # Ограничение градиентов для борьбы с взрывом градиентов
        'dropout_rate': 0.1,                                  # Регуляризация через дропаут
        'scheduler_type': 'cosine',                           # Тип расписания: 'linear', 'cosine', 'warmup'
        'warmup_steps': 100,                                  # Шаги разогрева для адаптации скорости обучения
        'accumulation_steps': 2,                              # Аккумуляция градиентов для виртуального батча
        'pos_weight': 'balanced',                             # Веса для дисбаланса классов ('balanced' или массив)
        'threshold': 0.4,                                     # Оптимальный порог классификации (можно подобрать на валидации)
        'use_fp16': True,                                     # Использование смешанной точности для ускорения
        'num_workers': 8,                                     # Параллелизм загрузки данных
        'label_smoothing': 0.1,                               # Сглаживание меток для борьбы с переобучением
        'layerwise_lr_decay': 0.85,                           # Дифференцированный LR по слоям BERT
        'freeze_layers': 6,                                   # Количество замороженных нижних слоев BERT
    }

    
    
    # ================== ИНИЦИАЛИЗАЦИЯ ==================
    print("\nНачало работы")
    print(f"Модель: {CONFIG['model_id']}")
    print(f"Данные: {CONFIG['data_path']}")
    print(f"Выходная директория: {CONFIG['output_dir']}")
    
    # Создаем директорию для сохранения результатов
    output_dir = Path(CONFIG['output_dir'])
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # ================== ПОДГОТОВКА ДАННЫХ ==================
    # Загружаем и разбиваем данные на тексты и метки
    texts, labels, label_columns = load_data(CONFIG['data_path'])
    num_labels = len(label_columns)
    
    # Разделяем на обучающую и валидационную выборки
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42  # 20% данных для валидации
    )
    
    print(f"\nРазбиение данных:")
    print(f"Обучающая выборка: {len(train_texts)} записей")
    print(f"Валидационная выборка: {len(val_texts)} записей")
    
    # ================== МОДЕЛЬ И ТОКЕНИЗАТОР ==================
    print("\nИнициализация токенизатора и модели...")
    # Загружаем предобученный токенизатор
    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_id'])
    # Инициализируем модель с нужным количеством меток
    model = MultilabelClassifier(CONFIG['model_id'], num_labels)
    
    # ================== СОЗДАНИЕ ДАТАСЕТОВ ==================
    # Создаем объекты для работы с данными
    train_dataset = MultiLabelTextDataset(
        train_texts,
        train_labels,
        tokenizer,
        CONFIG['max_length']  # Ограничение длины последовательности
    )
    val_dataset = MultiLabelTextDataset(
        val_texts,
        val_labels,
        tokenizer,
        CONFIG['max_length']
    )
    
    # Создаем загрузчики данных с батчами
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['train_batch_size'],
        shuffle=True,  # Перемешиваем данные при обучении
        num_workers=4  # Параллельная загрузка данных
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['eval_batch_size'],
        num_workers=4
    )
    
    # ================== НАСТРОЙКА ОБУЧЕНИЯ ==================
    # Определяем устройство для обучения (GPU/CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nИспользуемое устройство: {device}")
    
    # Перемещаем модель на целевое устройство
    model.to(device)
    
    # Инициализируем оптимизатор AdamW с L2 регуляризацией
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CONFIG['learning_rate'],
        weight_decay=CONFIG['weight_decay']
    )
    
    # Создаем планировщик скорости обучения
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=1.0,  # Начальный множитель lr
        end_factor=0.0,    # Конечный множитель lr
        total_iters=CONFIG['num_epochs']  # Количество шагов изменения
    )
    
    # ================== ОБУЧЕНИЕ МОДЕЛИ ==================
    model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        num_epochs=CONFIG['num_epochs'],
        patience=CONFIG['patience'],
        output_dir=output_dir,
        label_columns=label_columns
    )
    
    # ================== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ==================
    # Сохраняем финальную версию модели
    print(f"\nСохранение финальной модели в {output_dir}/final_model")
    model.bert.save_pretrained(output_dir / "final_model")
    tokenizer.save_pretrained(output_dir / "final_model")
    
    # Сохраняем список меток для последующего использования
    label_columns_path = output_dir / "label_columns.json"
    print(f"Сохранение списка меток в {label_columns_path}")
    with open(label_columns_path, "w") as f:
        json.dump(label_columns, f)
    
    print("\nОбучение завершено!")

/home/username/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
main()


Начало работы
Модель: modernBERT-large-multilingual
Данные: data/dataset.csv
Выходная директория: modernbert-russian-multilabel-final

Загрузка данных из data/dataset.csv...
Загружено 35303 записей с 19 метками

Разбиение данных:
Обучающая выборка: 28242 записей
Валидационная выборка: 7061 записей

Инициализация токенизатора и модели...


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at modernBERT-large-multilingual and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([19]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Используемое устройство: cuda

Начало обучения:
--------------------------------------------------

Эпоха 1/5

Обучение:


Прогресс: 100%|██████████| 1766/1766 [33:38<00:00,  1.14s/it]



Валидация:


...


Общие метрики мультилейбл классификации:
--------------------------------------------------
hamming_loss        : 0.0176
subset_accuracy     : 0.7558
macro_precision     : 0.8904
macro_recall        : 0.7357
macro_f1            : 0.8007
micro_f1            : 0.8217
samples_f1          : 0.7199
jaccard_score       : 0.7068

Метрики по отдельным меткам:
--------------------------------------------------
оффлайн_преступления     : precision: 0.7283, recall: 0.5055, f1: 0.5968
онлайн_преступления      : precision: 0.8926, recall: 0.6584, f1: 0.7578
наркотики                : precision: 0.9588, recall: 0.9257, f1: 0.9420
азартные_игры            : precision: 0.9486, recall: 0.8667, f1: 0.9058
порнография              : precision: 0.8821, recall: 0.5881, f1: 0.7057
проституция              : precision: 0.9127, recall: 0.8394, f1: 0.8745
рабство                  : precision: 0.8908, recall: 0.7828, f1: 0.8333
самоубийство             : precision: 0.9648, recall: 0.8949, f1: 0.9286
терроризм 

Прогресс: 100%|██████████| 1766/1766 [32:05<00:00,  1.09s/it]



Валидация:


Прогресс: 100%|██████████| 221/221 [02:48<00:00,  1.31it/s]
/home/username/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/username/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Общие метрики мультилейбл классификации:
--------------------------------------------------
hamming_loss        : 0.0172
subset_accuracy     : 0.7612
macro_precision     : 0.8995
macro_recall        : 0.7347
macro_f1            : 0.8031
micro_f1            : 0.8249
samples_f1          : 0.7206
jaccard_score       : 0.7082

Метрики по отдельным меткам:
--------------------------------------------------
оффлайн_преступления     : precision: 0.7682, recall: 0.4617, f1: 0.5768
онлайн_преступления      : precision: 0.8874, recall: 0.6634, f1: 0.7592
наркотики                : precision: 0.9706, recall: 0.9204, f1: 0.9449
азартные_игры            : precision: 0.9194, recall: 0.9012, f1: 0.9102
порнография              : precision: 0.8864, recall: 0.5762, f1: 0.6984
проституция              : precision: 0.9355, recall: 0.8153, f1: 0.8712
рабство                  : precision: 0.8798, recall: 0.8131, f1: 0.8451
самоубийство             : precision: 0.9653, recall: 0.9058, f1: 0.9346
терроризм 

# 📊 Отчет по анализу обучения модели

## 📌 Резюме

### Статус метрик:
- **Macro F1**: `0.803` ✅  
- **Micro F1**: `0.825` ✅  
- **Subset accuracy**: `0.761` ⚠️  
- **Hamming loss**: `0.017` ✅  

---

## 📈 1. Динамика обучения

| Метрика       | Изменение               | Статус  |
|---------------|-------------------------|---------|
| **Train loss** | `0.085` → `0.0017` (↓ 50x) | ✅      |
| **Val loss**   | `0.065` → `0.069` (↑ 6%)   | ⚠️      |

---

## 🎯 2. Качество по классам

### Распределение F1-score

#### Лучшие (>0.90):
- **нейтральный**: `0.973`  
- **религия**: `0.962`  
- **наркотики**: `0.945`  
- **самоубийство**: `0.935`  

#### Проблемные (<0.60):
- **политика**: `0.546`  
- **оффлайн_преступления**: `0.577`  
- **социальная_несправедливость**: `0.560`  

---

## ⚠️ 3. Ключевые проблемы

### Критические issues:
1. **Переобучение**: значительное расхождение `train/val loss`.  
2. **Дисбаланс**: разброс F1-score (`0.546` - `0.973`).  
3. **Стагнация**: валидационные метрики не улучшаются после эпохи 2.  

---

## 📋 4. План действий

### Приоритет 1: Остановка переобучения
- **Early stopping** на эпохе 3.  
- Увеличение регуляризации (например, добавление dropout или L2-регуляризации).  

### Приоритет 2: Балансировка классов
- Аугментация данных для редких классов.  
- Использование взвешенной функции потерь для учета дисбаланса.  

---

## 📝 Итоги

### Заключение
✅ **Позитивные моменты:**
- Хорошие общие показатели (Macro F1, Micro F1).  
- Стабильный рост качества на обучающей выборке.  

⚠️ **Требуют внимания:**
- Переобучение (расхождение между train и val loss).  
- Дисбаланс классов (разброс F1-score).  
- Ранняя остановка обучения из-за стагнации метрик.  

---

<p align="center">🚀 Продолжаем улучшать модель!</p>